In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader, random_split
from torchinfo import summary
import torchvision.models as torch_models

import onnx
import onnxruntime

import io
import os
import json
import base64
import random
from PIL import Image

In [2]:
# dataset

dataset_path = '/home/wyundi/Server/Courses/CS546/project/data/cat/train'

## Model

In [3]:
# device

def set_device(net, device='GPU'):
    if device == 'GPU':
        torch_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    else:
        torch_device = torch.device("cpu")

    if torch.cuda.device_count() > 1:
        model = nn.DataRarallel(model)

    net = net.to(torch_device)
    return net, torch_device

In [4]:
class dataset(Dataset):
    def __init__(self, path):
        
        self.__dataset_path = path
        
        self.__transforms = torch.nn.Sequential(
            T.Resize((224, 224)),
            T.RandomResizedCrop(224),
            T.RandomHorizontalFlip(),
        )
        
        self.data_list = []
        self.label_list = []
        
        for filename in os.listdir(path):
            self.__img_path = path + '/' + filename
            
            self.data_list.append(self.__img_path)
            self.label_list.append(0 if filename.split('.')[0] == 'cat' else 1)
            
        self.data = list(zip(self.data_list, self.label_list))
        
    def __getitem__(self, index):
        
        self.__img_path, self.__img_label = self.data[index]
        
        self.__img = Image.open(self.__img_path)
        self.__img = self.__transforms(self.__img)
        self.__img_np = np.array(self.__img)
        
        self.__img_label = torch.tensor(self.__img_label, dtype=torch.long)
        self.__img_tensor = torch.tensor(self.__img_np.transpose(2, 0, 1), dtype=torch.float32)
        
        self.__mean, self.__std = self.__img_tensor.mean(), self.__img_tensor.std()
        self.__std = 1e-03 if self.__std == 0 else self.__std
    
        self.__transforms_norm = torch.nn.Sequential(
            T.Normalize(self.__mean, self.__std)
        )

        self.__img_tensor = self.__transforms_norm(self.__img_tensor)
        
        return (self.__img_tensor, self.__img_label)
    
    def __len__(self):
        return len(self.data)

    def get_name(self):
        return self.name_list
        
cat_dataset = dataset(dataset_path)
dl = DataLoader(cat_dataset, batch_size=1, shuffle=True)

# for step, (inputs, labels) in enumerate(dl):
#     pass

In [5]:
class Convnet(nn.Module):
    def __init__(self, num_classes=1000):
        super(Convnet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Flatten(start_dim=1, end_dim=3)
        )
        self.classifier = nn.Sequential(
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(4096),
            nn.Linear(4096, 2048),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(2048),
            nn.Linear(2048, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x




Net = Convnet(2)
input_shape = (16, 3, 224, 224)

print(Net)

print(summary(Net, input_shape))

Convnet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Flatten(s

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,16,kernel_size=3, padding=0,stride=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(16,32, kernel_size=3, padding=0, stride=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
            )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(32,64, kernel_size=3, padding=0, stride=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        
        self.fc1 = nn.Linear(3*3*64,10)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(10,2)
        self.relu = nn.ReLU()
        
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1)
        out = self.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [7]:
# helper function for classification

def calc_acc(out, labels):
    num = out.size(0)
    prediction = out.argmax(dim=1)
    return (prediction == labels).sum().item()/num

def evaluate(model, torch_device, loss_func, dataloader, method='classification'):
    loss = 0
    acc = 0

    loss_list = []
    acc_list = []
    
    with torch.no_grad():
        for step, (inputs, labels) in enumerate(dataloader):
            
            inputs = inputs.to(torch_device)
            labels = labels.to(torch_device)

            out = model(inputs)
            loss_list.append(loss_func(out, labels))

            if method == 'classification':
                acc_list.append(calc_acc(out, labels))

        loss = torch.mean(torch.tensor(loss_list))

    if method == 'classification':
        acc = torch.mean(torch.tensor(acc_list, dtype=torch.float32))
        return loss, acc
    elif method == 'regression':
        return loss


In [8]:
# Define a train function

def train_model(model, train_dataset, test_dataset, batch = 256, epochs=50,
                lr=0.005, class_weights = None, weight_decay = 0):

    # train_history
    train_history = {}
    train_history['train_loss'] = []
    train_history['train_acc'] = []

    train_history['test_loss'] = []
    train_history['test_acc'] = []

    # set device
    model, torch_device = set_device(model)
    
    if class_weights != None:
        class_weights = class_weights.to(torch_device)

    # Dataloader
    train_dl = DataLoader(train_dataset, batch_size=batch, shuffle=True, drop_last=True)
    test_dl = DataLoader(test_dataset, batch_size=batch, shuffle=True, drop_last=True)
    
    # optimzer and loss_func
    optimzer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=weight_decay, amsgrad=False)
    loss_func = nn.CrossEntropyLoss(weight = class_weights)

    # train_process
    for epoch in range(epochs):
        
        for step, (inputs, labels) in enumerate(train_dl):
            
            inputs = inputs.to(torch_device)
            labels = labels.to(torch_device)

            out = model(inputs)
            loss = loss_func(out, labels)
            optimzer.zero_grad()
            loss.backward()
            optimzer.step()
            
            # output
            train_loss, train_acc = evaluate(model, torch_device, loss_func, train_dl)
            test_loss, test_acc = evaluate(model, torch_device, loss_func, test_dl)

            print(  'Epoch:', epoch+1, '/', epochs, ', '\
                    'train_loss: {loss:.5f}, '.format(loss = train_loss), \
                    'train_acc: {acc:.5f}, '.format(acc = train_acc), \
                    'test_loss: {loss:.5f}, '.format(loss = test_loss), \
                    'test_acc: {acc:.5f}'.format(acc = test_acc))

            train_history['train_loss'].append(train_loss)
            train_history['train_acc'].append(train_acc)

            train_history['test_loss'].append(test_loss)
            train_history['test_acc'].append(test_acc)
  
    return train_history

In [9]:
cat_dataset = dataset(dataset_path)

split_rate = 0.9
train_size = int(split_rate * len(cat_dataset))
test_size = len(cat_dataset) - train_size

print(test_size)

train_dataset, test_dataset = random_split(cat_dataset, [train_size, test_size])

batch = 1024
epochs = 1
lr = 0.003

2500


In [10]:
"""
Train Model
"""

# load model
# model = Convnet(2)
model = CNN()

model_save_path = '/home/wyundi/Server/Courses/CS546/project/repo/M_Hub/src/ipynb/cat.pth'
if os.path.exists(model_save_path):
    model.load_state_dict(torch.load('/home/wyundi/Server/Courses/CS546/project/repo/M_Hub/src/ipynb/cat.pth'))
print(summary(model, (batch, 3, 224, 224)))

# train
hist = train_model(model, train_dataset, test_dataset, batch, epochs, lr)

Layer (type:depth-idx)                   Output Shape              Param #
CNN                                      --                        --
├─Sequential: 1-1                        [1024, 16, 55, 55]        --
│    └─Conv2d: 2-1                       [1024, 16, 111, 111]      448
│    └─BatchNorm2d: 2-2                  [1024, 16, 111, 111]      32
│    └─ReLU: 2-3                         [1024, 16, 111, 111]      --
│    └─MaxPool2d: 2-4                    [1024, 16, 55, 55]        --
├─Sequential: 1-2                        [1024, 32, 13, 13]        --
│    └─Conv2d: 2-5                       [1024, 32, 27, 27]        4,640
│    └─BatchNorm2d: 2-6                  [1024, 32, 27, 27]        64
│    └─ReLU: 2-7                         [1024, 32, 27, 27]        --
│    └─MaxPool2d: 2-8                    [1024, 32, 13, 13]        --
├─Sequential: 1-3                        [1024, 64, 3, 3]          --
│    └─Conv2d: 2-9                       [1024, 64, 6, 6]          18,496
│    └─

In [11]:
save_path = '/home/wyundi/Server/Courses/CS546/project/repo/M_Hub/src/ipynb/cat.pth'
torch.save(model.state_dict(), save_path)

## Export model to ONNX

In [12]:
model = model.to('cpu')
torch_model = model.eval()
print(torch_model)

# model input
x = torch.randn((1, 3, 224, 224), requires_grad=True)
torch_out = torch_model(x)

CNN(
  (layer1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=576, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=10, out_features=2, bias=True)
  (re

In [13]:
# Export model
onnx_path = '../onnx/cat.onnx'
torch.onnx.export(model, x, onnx_path, export_params=True, do_constant_folding=True,
                  input_names=['input'], output_names=['output'])

In [14]:
# test onnx model

onnx_model = onnx.load(onnx_path)
check_res = onnx.checker.check_model(onnx_model)

In [15]:
ort_session = onnxruntime.InferenceSession(onnx_path)
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
print(ort_inputs)
ort_outs = ort_session.run(None, ort_inputs)

np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

{'input': array([[[[-0.46013075,  0.16131048,  1.3411492 , ...,  0.47016093,
           0.6627378 ,  0.01738982],
         [ 1.393888  ,  2.077385  , -0.2751896 , ...,  0.96145153,
           0.30042148,  0.79800147],
         [ 0.05929932,  0.11774691,  1.064682  , ...,  1.0067812 ,
          -0.77511686,  0.18467359],
         ...,
         [ 0.3249645 , -0.09760758, -0.8661106 , ...,  0.5851463 ,
          -0.31697223,  1.4835749 ],
         [-0.23666707, -0.0062504 , -0.07259718, ...,  0.35776502,
           0.5893171 , -0.76672655],
         [ 0.78680843, -0.3816563 ,  1.0277336 , ..., -0.03134407,
          -0.04578843, -0.9389643 ]],

        [[-0.4381995 ,  0.44542983, -0.84105694, ..., -0.61878425,
          -0.7541752 , -0.22854093],
         [ 0.2826845 ,  0.01130121,  1.9391162 , ...,  0.44259194,
          -0.81699294,  0.20036478],
         [ 0.81456697, -0.97180915,  1.6943713 , ..., -1.3432666 ,
           0.8279068 , -0.7821024 ],
         ...,
         [ 0.57990557, -

In [16]:
print(torch_out)
print(ort_outs)

tensor([[ 0.9504, -1.2846]], grad_fn=<AddmmBackward0>)
[array([[ 0.95036775, -1.2846333 ]], dtype=float32)]


In [17]:
print(torch_out.size())

torch.Size([1, 2])
